# Real Estate Investment Opportunities in King County
Authors: Anton Smirnov, Tom Chapman

## Overview

The King County housing market is one of the hottest in the country, remaining resilient even through the recent decline in demand. For this project, we will utilize linear regression modelling to analyze sales data and provide recommendations to investors looking to get in on the fun.

## Business Problem/Stakeholders
 
We (Atoman Research) are based in King County and working on a project for a real estate investment trust. They would like us to use our area expertise to advise them on what sort of properties are likely to maximize the value of their investment. They have been tight-lipped about their plans for the properties they purchase, so we do not know if they intend to buy and hold, rennovate, demolish and rebuild, etc.

King County is an extremely hot real estate market. Zillow's home value index indicates the typical home price is $934,000, which is up 24.5 percent over the past year. Demand for King County real estate is high partly because several large employers in the area that provide highly compensated positions with excellent benefit packages, causing a higher than average number of wealthy buyers. This phenomenon is similar to the Bay Area in California, and the tri-state area surrounding New York City. 

Because we aim to maximize the value of our all of clients' investments, we aim to provide advice to a broad range of investors. Certainly a higher sale price is appealing, but we must also consider the risk that comes with large individual investments. Expensive homes have a limited pool of buyers, even in a wealthy locale like King County. We aim to identify property features that result in the higheest price per square foot across all properties in our data. This allows our clients to make investment decisions that match their specific fund sizes and risk tolerances.

# Data Understanding

For this analysis we utilized a dataset of King County sales records collected over a 12 month period. Although the data dates back to 2014-2015, the age of the data has no impact on the recommendations presented herein. Therefore we treat the data as if it were current. The dataset includes all of the satndard datapoints one would expect when describing home sales. 

In [ ]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics

In [ ]:
# Import Housing Data
simple_housing = pd.read_csv('../data/kc_house_data.csv')
simple_housing.describe()

In [ ]:
# Define & remove out-of-scope features of the data
complexities = ['id','date', 'view', 'sqft_above', 'sqft_basement', 'yr_renovated', 'lat', 'long', 'sqft_living15', 'sqft_lot15']
simple_housing.drop(columns=complexities, inplace=True)
simple_housing.head()

In [ ]:
# Exploratory Visualizations
import warnings
warnings.filterwarnings('ignore')
fig = plt.figure(figsize = (10,10))
ax = fig.gca()
simple_housing.hist(ax = ax);

All of our features suffer from some level of skewness and will need to be transformed to facilitate modelling.

In [ ]:
# scatter matrix
pd.plotting.scatter_matrix(simple_housing, figsize=(10,12));

# Data Preparation & Feature Engineering

Because we aim to maximize the value of our clients' investment, we do not believe home price is the most meaningful target variable. Certainly a higher sale price is appealing, but we must also consider the risk that comes with large individual investments. Expensive homes have a limited pool of buyers, even in a wealthy locale like King County. We will focus our analysis on the price per square foot of the properties in our data in order to allow our clients to make investment decisions that match their specific fund sizes and risk tolerances.

We elect to drop the one instance of a building grade 3, as this code signifies that the property fails to meet minimum building standards and could be unsafe..

In [ ]:
# Add price per square foot feature
simple_housing['price_sqft'] = simple_housing['price'] / simple_housing['sqft_living']

# drop components of price_sqft, and poor-graded structure & confirm no NaNs
simple_housing = simple_housing.drop(columns=['price', 'sqft_living'])
simple_housing = simple_housing[simple_housing['grade']!='3 Poor']

# Map Missing values for waterfront to "no"
# There are only 146 waterfront properties in the dataset, less than 1% of known data.
simple_housing['waterfront'].fillna(value='NO', inplace=True)
simple_housing.isna().sum()

In [ ]:
# Identify the single variable most correlated with price_sqft (in this data frame it's yr_built).
simple_housing.corr()

In [ ]:
# Train/test split
X = simple_housing.drop(columns=['price_sqft'])
y = simple_housing['price_sqft']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, test_size=0.33)

In [ ]:
# Encode Categorical Variables in Test & Train data.

# Identify catgeorical values & create new df with only the categorical columns.
categoricals = ['condition', 'grade', 'waterfront', 'zipcode']
X_train_cats = X_train[categoricals]
X_test_cats = X_test[categoricals]

# OneHot encoding for categorical variables in Train data. Drop first column to avoid perfect multicolinearity.
ohe = OneHotEncoder(drop='first')
X_train_cats_encoded = ohe.fit_transform(X_train_cats)
X_train_cats_df = pd.DataFrame(X_train_cats_encoded.todense(), columns=ohe.get_feature_names(categoricals))

# Drop original categorical columns, concatenate encoded columns.
X_train_data = X_train.drop(columns=categoricals)
X_train_encoded = pd.concat([X_train_data.reset_index(drop=True), X_train_cats_df.reset_index(drop=True)], axis=1)
X_train_encoded.info()

In [ ]:
# OneHot encoding for categorical variables in Test data.
ohe = OneHotEncoder(drop='first')
X_test_cats_encoded = ohe.fit_transform(X_test_cats)
X_test_cats_df = pd.DataFrame(X_test_cats_encoded.todense(), columns=ohe.get_feature_names(categoricals))
X_test_data = X_test.drop(columns=categoricals)
X_test_encoded = pd.concat([X_test_data.reset_index(drop=True), X_test_cats_df.reset_index(drop=True)], axis=1)
X_test_encoded.info()

### Data Transformation
We identified previously that our continuous variables all suffer from some level of skew, which we attempted to address with a log-transformation. The results of the transformation are visible in the below scatter matrix--bathrooms, bedrooms and sqft_lot are noticeably improved, while year built and floors remain skewed.  

In [ ]:
# log-transform non-normal features of Train Data
non_normal = ['bathrooms', 'bedrooms', 'floors', 'sqft_lot', 'yr_built']
train_preds_log = pd.DataFrame([])
train_preds_log[non_normal] = np.log(X_train_encoded[non_normal])
train_preds_log = pd.concat([train_preds_log.reset_index(drop=True), X_train_encoded.iloc[:,6:].reset_index(drop=True)], axis=1)

# log-transform non-normal features of Test Data
test_preds_log = pd.DataFrame([])
test_preds_log[non_normal] = np.log(X_test_encoded[non_normal])
test_preds_log = pd.concat([test_preds_log.reset_index(drop=True), X_test_encoded.iloc[:,6:].reset_index(drop=True)], axis=1)

# Visualize Impact of Logged Train Preds
pd.plotting.scatter_matrix(train_preds_log[non_normal], figsize=(10,12));

To facilitate easier comparison of our model coefficients we applied a standard scalar to our train and test data below.

In [ ]:
# Perform scaling of our training data utilizing StandardScaler
ss = StandardScaler()

X_train_scaled = ss.fit_transform(train_preds_log)
X_train_scaled_df = pd.DataFrame(X_train_scaled, index=train_preds_log.index, columns=train_preds_log.columns)

# Perform scaling of our test data utilizing StandardScaler
X_test_scaled = ss.fit_transform(test_preds_log)
X_test_scaled_df = pd.DataFrame(X_test_scaled, index=test_preds_log.index, columns=test_preds_log.columns)

# Modelling

Our modelling efforts included a dummy regressor model, a simple linear regression model, and two multiple linear regression models. The models are presented in order from simplest to most complex to demonstrate our iterative process.

### Dummy Regressor

We begin modelling efforts by creating a dummy regressor model to provide a baseline. 

In [ ]:
# Code Credit to geeksforgeeks.org. https://www.geeksforgeeks.org/dummy-regressor/
lm = LinearRegression().fit(X_train_scaled, y_train)
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train_scaled, y_train) 
lm_dummy_median = DummyRegressor(strategy = 'median').fit(X_train_scaled, y_train)
y_predict = lm.predict(X_test_scaled)
y_predict_dummy_mean = lm_dummy_mean.predict(X_test_scaled)
y_predict_dummy_median = lm_dummy_median.predict(X_test_scaled)

In [ ]:
print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(y_test, y_predict_dummy_mean)))

print("Median absolute error (dummy): {:.2f}".format(median_absolute_error(y_test, y_predict_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(y_test, y_predict_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(y_test, y_predict_dummy_median)))

### First Simple Model

Among our initial set of features, the year in which the house was built was the most correlated variable with price per square foot. The negative correlation might mean that buyers in King county value older houses at a higher rate than new-builds. We began our analysis by modelling the log-scaled and standardized year in which the houses were constructed against the price per square foot. Unsurprisingly, the model is quite weak wtih an R<sup>2</sup> value of 0.012. We will move from here to a multiple linear regression model that considers all of the variables in our test data. 

In [ ]:
# Add constant term/y-intercept to train data
X_train_const = sm.add_constant(X_train_scaled_df)
X_test_const = sm.add_constant(X_test_scaled_df)

In [ ]:
# Reshape y_train to facilitate modeling against X_train
y_train=y_train.values.reshape(-1,1)
model = sm.OLS(endog=y_train, exog=X_train_const['yr_built']).fit()
model.summary()

### First Multiple Linear Regression Model

For our first multiple linear regression model, we elected to remove zip codes from the train/test data due to the large number of features created by one hot encoding the zips. We will explore their inclusion in subsequent models.

In [ ]:
# Remove columns beginning with 'zip' from the train/test data and redefine our variables with 1, signifying the model.
cols = [c for c in X_train_const if c.lower()[:3] != 'zip']
X_train_const1 = X_train_const[cols]
X_test_const1 = X_test_const[cols]

In [ ]:
# First Multiple Linear Regression Model
model = sm.OLS(endog=y_train, exog=X_train_const1).fit()
model.summary()

R<sup>2</sup> value is much improved over our simple model, but still quite low. We will likely need to add additional features in order to improve the model. From our original dataset, zipcode is an appealing proxy for several attributes of a home, i.e. school district, proximity to amenities, etc. We will include the effect of zip code on price_sqft in our next model.

##### Homoskedasticity

In [ ]:
# Fit Linear Regression object on the train data and scatter plot residuals.

lr = LinearRegression()
lr.fit(X_train_const1, y_train)

y_hat = lr.predict(X_train_const1)
resid = y_train - y_hat

fig, ax=plt.subplots()
ax.scatter(x=range(y_hat.shape[0]), y=resid, alpha=0.5);

##### Normality of Errors
Creating a QQ plot and a histogram of the residuals revealed that although our transformations improved the normality of the data distribution, our train and test data contain outliers. Future models could explore removing the outliers in order to improve model accuracy.

In [ ]:
train_preds = lr.predict(X_train_const1)
test_preds = lr.predict(X_test_const1)

# Calculate Residuals
train_residuals = np.ndarray.flatten(y_train) - np.ndarray.flatten(train_preds)
test_residuals = y_test - np.ndarray.flatten(test_preds)

# QQ Plot -- Train
sm.qqplot(train_residuals, line='r');

In [ ]:
# QQ Plot -- Test
sm.qqplot(test_residuals, line='r');

In [ ]:
# Histogram of residuals
plt.hist(train_residuals, label='Train')
plt.hist(test_residuals, label='Test')
plt.legend()

plt.show()

##### Multicolinearity
Building grade 7 and grade 8 are above the general comfort level for VIF of 5. Future models could look to address this feature, although as categorical variables we cannot remove one or the other. 

In [ ]:
# Test for multicolinearity among features with VIF score

vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(X_train_const1.values, i) for i in range(len(X_train_const1.columns))]
vif["features"] = X_train_const1.columns

vif.sort_values(by='VIF', ascending=False)

#### Model Validation

Our first model performed slightly worse on the test than train data, but not enough to merit concern. Train/Test root mean squared errors appear quite high at 90.33 / 91.23 per square foot.

In [ ]:
# Fit linear regression object on train data
regressor = LinearRegression()
regressor.fit(X_train_const1, y_train)

# Calculate predicted values for train and test data.
y_pred_train = regressor.predict(X_train_const1)
y_pred_test = regressor.predict(X_test_const1)

# Calculate mean absolute and squared error and root mean squared error on TRAIN data
train_mae = mean_absolute_error(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)
train_rmse = np.sqrt(train_mse)

print(f'Train mean absolute error: {train_mae:.2f}')
print(f'Train mean squared error: {train_mse:.2f}')
print(f'Train root mean squared error: {train_rmse:.2f}')

# Calculate mean absolute and squared error and root mean squared error on TEST data
test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)
test_rmse = np.sqrt(test_mse)

print(f'Test mean absolute error: {test_mae:.2f}')
print(f'Test mean squared error: {test_mse:.2f}')
print(f'Test root mean squared error: {test_rmse:.2f}')

### 2nd Linear Regression - Zip Codes

Including the zip code data has dramatically improved the R<sup>2</sup> value of our model--slightly more than doubling it from 0.323 to 0.665. It appears that zip codes generally have a significant impact on the price per square foot of the properties in the data set.

There are several large pvalues within the model, suggesting that number of floors, building grade 4, and certain zip codes may not have a stastically significant impact on the price per square foot of those properties. Given more time or resources we would look to investigate these features to better address their impact on the model.

In [ ]:
# Rerun multiple linear regression, this time including zip codes
model = sm.OLS(endog=y_train, exog=X_train_const).fit()
model.summary()

#### Checking Model Assumptions

##### Homoskedasticity

A scatter plot of our model's residuals did not reveal any concerns with heteroskedasticity. There is not a clearly observable cone pattern that would make us think our data's errors are not evenly distributed.

In [ ]:
# Fit Linear Regression object on the train data and scatter plot residuals.

lr = LinearRegression()
lr.fit(X_train_const, y_train)

y_hat = lr.predict(X_train_const)
resid = y_train - y_hat

fig, ax=plt.subplots()
ax.scatter(x=range(y_hat.shape[0]), y=resid, alpha=0.5);

##### Normality
Creating a QQ plot and a histogram of the residuals revealed that although our transformations improved the normality of the data distribution, our train and test data contain outliers. Future models could explore removing the outliers in order to improve model accuracy.

In [ ]:
train_preds = lr.predict(X_train_const)
test_preds = lr.predict(X_test_const)

# Calculate Residuals
train_residuals = np.ndarray.flatten(y_train) - np.ndarray.flatten(train_preds)
test_residuals = y_test - np.ndarray.flatten(test_preds)

# QQ Plot -- Train
sm.qqplot(train_residuals, line='r');

In [ ]:
# QQ Plot -- Test
sm.qqplot(test_residuals, line='r');

In [ ]:
# Histogram of residuals
plt.hist(train_residuals, label='Train')
plt.hist(test_residuals, label='Test')
plt.legend()

plt.show()

##### Multicolinearity
Building grade 7 and grade 8 remain problematic, and their impacts were magnified slightly in this model. Given additional time and resources, we would explore whether removing these features would improve model accuracy. 

In [ ]:
# Test for Multicolinearity

vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(X_train_const.values, i) for i in range(len(X_train_const.columns))]
vif["features"] = X_train_const.columns

vif.sort_values(by='VIF', ascending=False)

### Final Model Validation / Evaluation

In this section, we will discuss the performance and validity of our model, as well as the impact of certain features with the largest correlation coefficients. Note that some of the data referenced here appears below the written work, specifically the train/test comparisons of R<sup>2</sup> and RMSE.

With an R<sup>2</sup> score of 0.665, we can say that our model captures ~66% of the variance in price per square foot that our features can collectively explain. As noted above, there are small issues with multicolinearity that could be addressed in future iterations to improve the model. Encouragingly, our model performed very similarly on our test and train data, with no significant difference in either  R<sup>2</sup> or RMSE scores. This suggests that the model is functioning with relatively low bias and low variance. While the absolute R<sup>2</sup> score leaves much to be desired, our model does not seem to be under or overfitted. Additional features could be added to improve R<sup>2</sup>.

The root mean squared error score of 63.94 on the test data signifies that our model will generally have error of \\$63.94 per square foot of a property. Compared with the mean price per square foot in the test data of $264.29, a \\$63.94 error is significant, particularly when properties can be several thousand square feet. However, the RMSE is well within one standard deviation of the price per square foot in the test data, which measured \\$110.42. Comparisons to the train data were similar, with a root mean squared error score of 63.53, compared to mean price per square foot of \\$264.05 and a standard deviation of \\$109.78.

Our 12 strongest coefficients (ranging from 37.01 for zip 98004 to 24.39 for zip 98119) are all related to the zip codes in which the properties are located, suggesting that the attributes of a zip code have significant impact on property value. Following these appealing zip codes, building grade 7, which was labeled 'average' carried the next largest impact on price per square foot, with a negative coefficient of -24.12. Perhaps unsurprisingly, the waterfront feature had a coefficient of 19.63, suggesting that a waterfront property can expect an increase of \\$19.63 per square foot.

Based on our evaluation of model performance, we believe its use is warranted provided further effort goes into reducing outliers in the data.

In [ ]:
print('R-squared score on train data: ' ,lr.score(X_train_const, y_train).round(4))
print('R-squared score on test data: ' , lr.score(X_test_const, y_test).round(4))

In [ ]:
# Fit linear regression object on train data
regressor = LinearRegression()
regressor.fit(X_train_const, y_train)

# Calculate predicted values for train and test data.
y_pred_train = regressor.predict(X_train_const)
y_pred_test = regressor.predict(X_test_const)

# Calculate mean absolute and squared error and root mean squared error on TRAIN data
train_mae = mean_absolute_error(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)
train_rmse = np.sqrt(train_mse)

print(f'Train mean absolute error: {train_mae:.2f}')
print(f'Train mean squared error: {train_mse:.2f}')
print(f'Train root mean squared error: {train_rmse:.2f}')

# Calculate mean absolute and squared error and root mean squared error on TEST data
test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)
test_rmse = np.sqrt(test_mse)

print(f'Test mean absolute error: {test_mae:.2f}')
print(f'Test mean squared error: {test_mse:.2f}')
print(f'Test root mean squared error: {test_rmse:.2f}')

In [ ]:
# Calculate price_sqft mean and standard deviation in train/test data.
print('Mean price per square foot in training data: ', y_train.mean().round(2))
print('Mean price per square foot in test data: ', y_test.mean().round(2))

print('Standard deviation of price per square foot in training data: ', y_train.std().round(2))
print('Standard deviation of price per square foot in test data: ', y_test.std().round(2))

## Visualizations

The following visualizations were created to aid in the presentation and highlight meaningful coefficients from our model.

In [ ]:
# Create dataframe of correlation coefficients from Final Model.
regressor = LinearRegression()
regressor.fit(X_train_const, y_train)
model_coefficients = regressor.coef_.reshape(-1,1)
feature_names = X_train_const.columns
coefficients_df = pd.DataFrame(data = model_coefficients, index = feature_names, columns = ['Coefficient value'])
coefficients_df2 = coefficients_df.sort_values(by='Coefficient value', ascending=False).head(25)

In [ ]:
waterfront_coef = [19.63220]

In [ ]:
# Create dataframe of mean data for zip codes with the eight highest correlation coefficients
simple_means = simple_housing.groupby('zipcode').mean()
target_codes = [98004, 98103, 98115, 98117, 98033, 98105, 98040, 98199] 
viz_zips = simple_means.loc[target_codes].reset_index()
viz_zips

#### Mean Price per Square Foot by Zip Code for Zips with Highest Impact on Price per Square Foot

In [ ]:
# Create x and y axis label data
zip_labels = list(viz_zips['zipcode'].apply(str))
price_labels = list(viz_zips['price_sqft'].round(2))

# Plot mean price/sqft for zip codes in zip labels
plt.rcParams.update({'font.size': 15});
x = np.arange(len(zip_labels))
width = 0.35 

fig, ax = plt.subplots(figsize=(15,12))
ax.set_ylabel('Price per Square Foot($)')
ax.set_xlabel('Zip Code')
ax.set_title('Mean Price per Square Foot by Zip Code')
ax.set_xticks(x)
ax.set_xticklabels(zip_labels)

pps = ax.bar(zip_labels, price_labels, color='green')
for p in pps:
   height = p.get_height()
   ax.annotate('{}'.format(height),
      xy=(p.get_x() + p.get_width() / 2, height),
      xytext=(0, 5), 
      textcoords="offset points",  
      ha='center', va='bottom')
plt.tight_layout()

plt.savefig('../images/mean_psft_zip', facecolor='white', dpi=500)

#### Correlation Coefficients for Zip Codes with Highest Impact on Price per Square Foot

In [ ]:
# Create x and y axis label data
zip_labels = list(viz_zips['zipcode'].apply(str))
viz_coef = coefficients_df2['Coefficient value'].head(8).round(2).to_list()


plt.rcParams.update({'font.size': 15});
x = np.arange(len(zip_labels)) 
width = 0.35 

fig, ax = plt.subplots(figsize=(15,12))

ax.set_ylabel('Correlation Coefficient')
ax.set_xlabel('Zip Code')
ax.set_title('Zip Codes Most Correlated with Increased Price per Square Foot')
ax.set_xticks(x)
ax.set_xticklabels(zip_labels)

pps = ax.bar(zip_labels, viz_coef, color='green')
for p in pps:
   height = p.get_height()
   ax.annotate('{}'.format(height),
      xy=(p.get_x() + p.get_width() / 2, height),
      xytext=(0, 5), # 3 points vertical offset
      textcoords="offset points",  
      ha='center', va='bottom')
plt.tight_layout()
plt.savefig('../images/zip_coefs', facecolor='white', dpi=500)

#### Correlation Coefficients for Building Grades

In [ ]:
# Create x and y axis label data

grade_coefs =  {'4': 0.450,
                '5': -4.17,
                '6': -14.45,
                '7': -24.12,
                '8': -17.44,
                '9': -6.35,
                '11': 3.02,
                '13': 3.19,
                '12': 4.13}

plt.rcParams.update({'font.size': 15});
x = np.arange(len(grade_coefs.keys())) 
width = 0.35 

fig, ax = plt.subplots(figsize=(15,12))

ax.set_ylabel('Correlation Coefficient')
ax.set_xlabel('Building Grade')
ax.set_title('Building Grade Correlation with Price per Square Foot')
ax.set_xticks(x)
ax.set_xticklabels(grade_coefs.keys())

mask1 = y < 0
mask2 = y >= 0
pps = ax.bar(grade_coefs.keys(), grade_coefs.values(), color='green')

for p in pps:
    height = p.get_height()
    if height > 0:
        ax.annotate('{}'.format(height), xy=(p.get_x() + p.get_width() / 2, height), xytext=(0, 15),
        textcoords="offset points", ha='center')  
        
    else:
        ax.annotate('{}'.format(height), xy=(p.get_x() + p.get_width() / 2, height), xytext=(0, -15),
        textcoords="offset points", ha='center')  # 3 points vertical offset
plt.tight_layout()
plt.savefig('../images/grade_coefs', dpi=500)

#### Final Model Image for README

In [ ]:
#Code credit to sascha @ stackoverflow. https://stackoverflow.com/questions/46664082/python-how-to-save-statsmodels-results-as-image-file

plt.rc('figure', figsize=(7, 7), facecolor='white')
#plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 12}) old approach
plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 10}, fontproperties = 'monospace') # approach improved by OP -> monospace!
plt.axis('off')

plt.tight_layout()
plt.savefig('../images/model', bbox_inches = 'tight', facecolor='white', dpi=500)

In [ ]:
coefficients_df.sort_values(by = 'Coefficient value', ascending=False).head(25)

# Conclusions

Our final model performed similarly on the train and test data, and with an R<sup>2</sup> score of 0.66, it is approaching a usable model. Further iteration is needed to address small issues with multicolinearity and address the presence of outliers in both the train and test data. We identified the following features and their correlation coefficients that had the strongest positive or negative impacts on price per square foot in King County:

* __Zip Codes 98004 (37.01), 98103 (35.27), 98115 (31.76), 98117 (30.79), 98112 (30.50),  98033 (26.22), 98105 (25.90), and 98040 (25.32).__ Considering that school districts, traffic patterns, proximity to amenities, crime, etc. all vary with zip code, it is reasonable to expect that the difference between our baseline zip code of 98001 and certain others will have a significant impact on property value. The baseline zip of 98001 is primarily associated with Auburn, WA, an inland suburb situated about halfway between Seattle and Tacoma. We chose this as a baseline because it is close enough to have access to Seattle-area amenities but far enough away to be more reasonably priced than some of the urban areas.

* __Building Grades 7 (-24.12) and 8 (-17.44).__ While building grades are quite subjective, these two grades most closely define average quality of construction methods and materials. Each has a negative impact on price per square foot, with grade 7 and 8 having correlation coefficients of -24.12 and -17.44, respectively. One might assume that given the climate, proximity to salt water and steep hills present in King County, buyers prize quality materials and construction methods that will last in a hostile environment.

* __Bedrooms (-20.82).__ All else being equal, each additional bedroom had a negative effect on price per square foot of \\$20.81. At first blush this is surprising, but square footage that is used for bedrooms is not being utilized for spaces that are more closely associated with home value such as kitchen and entertaining space. Further analysis would be needed to more fully explain the negative correlation of bedrooms and price per square foot in King County.

* __Waterfront Property (19.63).__ Perhaps the least surprising finding in the study, a property's designation as waterfront had a positive impact on price per square foot. 


# Recommendations

Based on our findings, we recommend that:
1. Focus property searches within the zip codes with most impact on price per square foot.
2. Avoid properties built with average materials and techniques. 
3. Be thoughtful about the number of bedrooms in target properties. Unless you have requirements for a high number of bedrooms, it would be best to avoid properties that use space this way.
4. To the extent waterfront property is available and priced appealingly, it should be the first stop for investors in King County.